# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.stats.api as sms

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [5]:
w = data[data.race=='w']
b = data[data.race=='b']
wc = sum(data[data.race=='w'].call)
bc = sum(data[data.race=='b'].call)
print(len(data))
print(len(b))
print(len(w))
print(wc)
print(bc)

4870
2435
2435
235.0
157.0


We can use the Z test if the data is coming from normal distribution. CLT will allow us to assume that the 2 sample proportion data is from normal distribution. Since it is a large sample size so CLT applies and we can use the Z-test

Question 2:
H0: Black Calls - White Calls = 0 OR Black Calls = White Calls 
H1: Black Calls - White Calls != 0 OR Black Calls != White Calls 

In [6]:
# Your solution to Q3 here
#Testing with two sample proportion Z test

# p1 is the callback rate for the white-sounding names
p1 = sum(data.call[data.race=='w'])/sum(data.race=='w')
n1 = sum(data.race=='w')

# p2 is the callback rate for the black-sounding names
p2 = sum(data.call[data.race=='b'])/sum(data.race=='b')
n2 = sum(data.race=='b')

prop_diff = p1-p2
pc = (n1 * p1 + n2 * p2)/ (n1 + n2)
sigma = np.sqrt(pc*(1-pc)/n1 + pc*(1-pc)/n2)

print('prop_diff: ', prop_diff)
print('standard deviation: ', sigma)

# generic formula to get z score:  (test statistic - hypothesized value)/ (stand. deviation of statistic)
z = (p1-p2 -0) / sigma
print('\nz score: ', z)

# p value area of the curve for the specified z value
# (1 - (area of curve above z value) * 2
p = (1 - stats.norm.cdf(z)) * 2
print('p value: ', p)


# z_critical at 95%
z_crit = stats.norm.ppf(q=0.975)
print('z_critical: ', z_crit)

# margin of error
moe = z_crit * sigma
print('\nmargin of error: ', moe)

# confidence interval
ci = (prop_diff-moe, prop_diff+moe)
print('95% confidence interval: ', ci)

prop_diff:  0.032032854209445585
standard deviation:  0.007796894036170457

z score:  4.108412152434346
p value:  3.983886837577444e-05
z_critical:  1.959963984540054

margin of error:  0.015281631502169232
95% confidence interval:  (0.01675122270727635, 0.04731448571161482)


In [9]:
#BOOTSTRAP METHOD 
# We can resample from the same population assuming that there is no difference between the two proportions.
def get_prop_diff(s1, s2):
    
    p1 = np.sum(s1['call'] == 1)/len(s2)
    p2 = np.sum(s1['call'] == 1)/len(s2)
    
    return abs(p1-p2)
    
def get_bs_samples_diff(s1, s2, func, size):
    length1 = len(s1)
    length2 = len(s2)
    bs_prop_diffs = np.empty(size)
    
    for i in range(size):
        combined_sample = pd.concat([s1,s2])
        shuffled_sample = combined_sample.sample(length1+length2).reset_index(drop=True)

        new_sample1 = shuffled_sample.iloc[:length1,:]
        new_sample2 = shuffled_sample.iloc[length1:,:]
        
        bs_prop_diffs[i] = func(new_sample1,new_sample2)
        
    return bs_prop_diffs

bs_samples_diff = get_bs_samples_diff(data[data.race=='w'], data[data.race=='b'], get_prop_diff, 200)

p = np.sum(bs_samples_diff > prop_diff)/len(bs_samples_diff)
print('number of times the random selected prop differences is greater than our samples prop diff is: ', p)

number of times the random selected prop differences is greater than our samples prop diff is:  0.0


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Question 4:
H0 will be rejected. Yes there is significance difference between black and white name sounding resumes. Using 95% confidence level as threshold, there is 95% confidence that confidence interval of "difference in % callbacks in black and white sounding resume" is from 0.017 to 0.047

Question 5: 
The analysis I did does not mean that race/name is the most important factor in callback success. It only means that race/name is a factor that affect callback success. Correlation between different features and callback success must be analyzed and ranked to test which feature is most important in callback success.